<a href="https://colab.research.google.com/github/Vsevolod245/tensor-contraction/blob/master/Peshikov/PeshikovVE_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Имеются 5 тензеров: А, B, C, D, E.<br>
Задача: осуществить свертку и выполнить ее З-мя способами:<br>
1) свернуть, используя цикл for<br>
2) вручную используя функции reshape<br>
3) используя пакет ncon<br><br>

Сравнить эти способы по:<br>
а) объему написания кода<br>
б) по времени исполнения<br>
в) проверить, что эти способы являются равнозначными (сравнения тензеров можно произвести с помощью пакета numpy)<br><br>
Итогом свертки будет некий тензор R валентности 3

In [ ]:
pip install ncon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import time
from ncon import ncon

Обявляем тензоры

In [ ]:
# Свободная нога
FreeLeg = 8
# Не Свободные ноги
Leg1 = 1
Leg2 = 2
Leg3 = 3
Leg4 = 4
Leg5 = 5
Leg6 = 6
Leg7 = 7

A = np.random.uniform(-1,1,(FreeLeg,Leg5 ,Leg1))
B = np.random.uniform(-1,1,(Leg5 ,FreeLeg,Leg6,Leg2))
C = np.random.uniform(-1,1,(Leg1,Leg2,Leg3,Leg4))
D = np.random.uniform(-1,1, (FreeLeg,Leg3,Leg7))
E = np.random.uniform(-1,1,(Leg7,Leg4,Leg6))

#1. С использованием цикла for

In [ ]:
TimeStart  = time.time()
# Свертка 1
SV1 = np.zeros((FreeLeg, Leg5, Leg2, Leg3, Leg4))
for freeLeg in range(FreeLeg):
  for leg5 in range(Leg5):
    for leg2 in range(Leg2):
      for leg3 in range(Leg3):
        for leg4 in range(Leg4):
          for sum in range(Leg1):
            SV1[freeLeg, leg5, leg2, leg3, leg4] = SV1[freeLeg, leg5, leg2, leg3, leg4] + A[freeLeg][leg5][sum] * C[sum][leg2][leg3][leg4]
# Свертка 2
SV2 = np.zeros((FreeLeg, Leg3, Leg4, FreeLeg, Leg6))
for freeLeg in range(FreeLeg):
  for leg3 in range(Leg3):
    for leg4 in range(Leg4):
      for freeLeg1 in range(FreeLeg):
        for leg6 in range(Leg6):
          for sum in range(Leg5):
            for sum1 in range(Leg2):  
              SV2[freeLeg, leg3, leg4, freeLeg1, leg6] = SV2[freeLeg, leg3, leg4, freeLeg1, leg6] + SV1[freeLeg, sum, sum1, leg3, leg4] * B[sum, freeLeg1, leg6, sum1]
# Свертка 3
SV3 = np.zeros((FreeLeg,Leg3,FreeLeg,Leg7))
for freeLeg in range(FreeLeg):
  for leg3 in range(Leg3):
    for freeLeg1 in range(FreeLeg):
      for leg7 in range(Leg7):
        for sum in range(Leg4):
          for sum1 in range(Leg6):
            SV3[freeLeg, leg3, freeLeg1, leg7] = SV3[freeLeg, leg3, freeLeg1, leg7] + SV2[freeLeg, leg3, sum, freeLeg1, sum1] * E[leg7, sum, sum1]
# Свертка 4
SV4  = np.zeros((FreeLeg,FreeLeg,FreeLeg))
for freeLeg in range(FreeLeg):
  for freeLeg1 in range(FreeLeg):
    for freeLeg2 in range(FreeLeg):
      for sum in range(Leg3):
        for sum1 in range(Leg7):
          SV4[freeLeg, freeLeg1, freeLeg2] =  SV4[freeLeg, freeLeg1, freeLeg2] + SV3[freeLeg, sum, freeLeg1, sum1] + D[freeLeg2, sum, sum1]

TimeEnd = time.time()
print('Total time "For" = ' + str(TimeEnd - TimeStart))
print(f"result_For.shape = {SV4.shape}")
forResult = SV4

Total time "For" = 0.11841344833374023
result_For.shape = (8, 8, 8)


#2. С использованием функции reshape

In [ ]:
TimeStart  = time.time()

# Свертка 1
SV1 = np.tensordot(A, C, axes = ([2], [0]))
# Свертка 2
SV2 = np.tensordot(SV1, B, axes = ([1, 2], [0, -1]))
# Свертка 3
SV3 = np.tensordot(SV2, E, axes = ([2, -1], [-2, -1]))
# Свертка 4
SV4 = np.tensordot(SV3, D, axes = ([1, -1], [-2, -1]))

TimeEnd = time.time()
print('Total time "reshape" = ' + str(TimeEnd - TimeStart))
print(f'result_Tensordot.shape = {SV4.shape}')
reshapeResult = SV4

Total time "reshape" = 0.0017459392547607422
result_Tensordot.shape = (8, 8, 8)


#3. C использованием пакета ncon

In [ ]:
TimeStart  = time.time()

VseSvertki = ncon(
    (A, B, C, D, E),
    ([-1, 2, 1], [2, -2, 5, 3], [1, 3, 6, 4], [-3, 6, 7], [7, 4, 5])
)

TimeEnd = time.time()
print('Total time "ncom" = ' + str(TimeEnd - TimeStart))
print(f'result_ncon.shape = {VseSvertki.shape}')
ncomResult = VseSvertki

Total time "ncom" = 0.006867885589599609
result_ncon.shape = (8, 8, 8)


# Проверка

# Сравнение

а)код: ncom < reshape < for<br>
б)время: reshape < ncom < for<br>

# в) сверка

In [ ]:
print(ncomResult-forResult)
print(ncomResult-reshapeResult)

[[[-1.20608210e+01 -1.78208794e+01 -1.64219292e+01 -2.46340276e+01
   -2.14985197e+01 -2.58914794e+01 -2.19032893e+01 -1.47879439e+01]
  [-1.77873169e+00 -2.24284175e+00  2.59706232e+00  1.05356357e+01
   -9.82976561e+00 -3.50943774e+00  5.95681993e-01 -5.34440771e+00]
  [-3.10131176e+00 -1.08019593e+01 -5.95332667e+00  7.50040009e-01
   -9.67109500e+00 -5.90361474e+00 -5.25790954e+00 -6.32843061e+00]
  [-9.64498432e+00 -9.12370165e+00 -8.36526973e+00 -1.12425726e+00
   -4.00581027e+00 -8.31951692e+00 -6.25923207e+00 -1.50047463e+01]
  [ 8.54882413e+00  4.00011447e-01  5.37171309e+00  1.05949137e+01
   -1.53386604e+00 -3.84904697e+00  7.67167710e+00 -3.51239119e-01]
  [-1.27240359e+01 -1.89959218e+01 -4.73536533e+00 -1.91097432e+00
   -9.24061879e+00 -1.49044282e+01 -1.40456782e+01 -1.70117144e+01]
  [ 2.32580050e+00  2.29269611e-01 -2.00653701e+00  8.46786310e+00
   -3.22356935e+00 -5.82235770e+00 -3.61261651e-01 -6.60515410e+00]
  [-5.51513551e+00 -6.73941861e+00 -1.50802737e+01 -2.0